In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline



In [15]:
model = "cardiffnlp/twitter-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForSequenceClassification.from_pretrained(model)

sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


c:\Users\k4ty2\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
label_mapping = {
    "LABEL_0": "negative",
    "LABEL_1": "neutral",
    "LABEL_2": "positive"
}


text = "I do not want do it!"
result = sentiment_pipeline(text)

label = result[0]['label'] 


mapped_label = label_mapping[label] 

print(mapped_label)
print(f"confidence - {result[0]['score']}")

negative
confidence - 0.9362080097198486
